In [40]:
import pickle

import pandas as pd
import numpy as np

In [41]:
basedir = "./pkls/"

##Get whether track should be used for recommendations or not

In [43]:
with open(basedir + "metas_csv.csv", 'r') as tf:
    metas = pd.read_csv(tf, index_col=0)

In [44]:
metas.head()

,track_id,song_hottt,artist_hottt,artist_fam
0,TRBGPHG12903CE6CC3,0.252446,0.357470,0.469672
1,TRBGPYK128F42796E1,NaN,0.305101,0.382720
2,TRBGPJP128E078ED20,NaN,0.610731,0.872537
3,TRBGPXH128F428C912,NaN,0.288050,0.404823
4,TRBGPSV12903CA9C25,NaN,0.412700,0.595409


In [66]:
#arbitrary bounds:
upperb = 1.0 #don't recommend people who are too popular (Kanye West, Daft Punk, etc have > 1 artist_hottt)
lowerb = 0.6 #don't recommend people who have no following
def recommend(row):
    if row["song_hottt"] > upperb or row["artist_hottt"] > upperb or row["artist_fam"] > upperb:
        return 0
    elif row["song_hottt"] >= lowerb or row["artist_hottt"] >= lowerb or row["artist_fam"] >= lowerb:
        return 1
    return 0

In [67]:
metas["recommend"] = metas.apply(recommend, axis=1)

In [77]:
metas.head(7)

,track_id,song_hottt,artist_hottt,artist_fam,recommend
0,TRBGPHG12903CE6CC3,0.252446,0.357470,0.469672,0
1,TRBGPYK128F42796E1,NaN,0.305101,0.382720,0
2,TRBGPJP128E078ED20,NaN,0.610731,0.872537,1
3,TRBGPXH128F428C912,NaN,0.288050,0.404823,0
4,TRBGPSV12903CA9C25,NaN,0.412700,0.595409,0
5,TRBGPAX12903CEC037,NaN,0.353036,0.519000,0
6,TRBGPZX128F4285105,0.530663,0.388753,0.626821,1


In [69]:
print len(metas[metas["recommend"] ==0].index)

5874


In [70]:
metas[metas["recommend"] ==0].tail(6)

,track_id,song_hottt,artist_hottt,artist_fam,recommend
9989,TRAQQUC128F9314732,0.591754,0.414011,0.565605,0
9991,TRAQQYH12903D0C595,0.537504,0.405354,0.414242,0
9994,TRAQQLM128F147CBFE,0.440187,0.331373,0.492670,0
9996,TRAQQUQ128F933671B,NaN,0.418736,0.515456,0
9997,TRAQQZN128F931FA31,0.450762,0.369870,0.537533,0
9998,TRAQQEE128EF342942,0.215080,0.432804,0.517481,0


In [71]:
recs = metas[["track_id", "recommend"]]
print len(recs.index)

10000


In [74]:
recs.head()

,track_id,recommend
0,TRBGPHG12903CE6CC3,0
1,TRBGPYK128F42796E1,0
2,TRBGPJP128E078ED20,1
3,TRBGPXH128F428C912,0
4,TRBGPSV12903CA9C25,0


##Get cluster prediction per track

In [78]:
def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [79]:
with open(basedir + "zscore_df_csv.csv", 'r') as tf:
    tracks = pd.read_csv(tf, index_col=0)

In [80]:
tracks.head(2)

,track_id,duration,fade_in,tempo,tatum_density,beat_density,segment_density,section_density,loudness,mode,...,timbre_mode3,timbre_mode4,timbre_mode5,timbre_mode6,timbre_mode7,timbre_mode8,timbre_mode9,timbre_mode10,timbre_mode11,timbre_mode12
0,TRBGPHG12903CE6CC3,-0.668187,-0.319412,-0.194199,-0.341662,-0.464571,-0.32573,1.088936,-1.007373,1,...,1.259628,1.189218,0.437165,0.945188,-0.284978,0.281554,0.185953,-0.112983,-1.104735,0.363289
1,TRBGPYK128F42796E1,0.926188,4.084562,0.881785,-0.346080,0.578074,0.36889,-1.047679,0.092725,1,...,0.536204,0.386584,0.661103,-0.212270,-1.066440,0.317366,-0.484924,0.358257,0.436883,0.239837


In [81]:
kmeans = unpickle(basedir + "kmeans40.pkl")

In [82]:
kmeans_new = unpickle(basedir + "kmeans40_None_2.pkl")

In [85]:
def predict(row):
    features = row.iloc[1:].tolist()
    return kmeans.predict(features)[0]

def predict_new(row):
    features = row.iloc[1:].tolist()
    return kmeans_new.predict(features)[0]

In [86]:
with open(basedir + "trackinfo_old.csv", 'r') as tof:
    info = pd.read_csv(tof, index_col=0)

In [87]:
info.head(2)

,track_id,title,artist,recommend,cluster
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),Bo Carter,0,23
1,TRBGPYK128F42796E1,Des Vôtres,Fredericks_ Goldman_ Jones,0,37


In [88]:
#tracks["cluster"] = tracks.apply(predict, axis=1)
info["cluster_new"] = tracks.apply(predict_new, axis=1)

In [89]:
info.head(2)

,track_id,title,artist,recommend,cluster,cluster_new
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),Bo Carter,0,23,33
1,TRBGPYK128F42796E1,Des Vôtres,Fredericks_ Goldman_ Jones,0,37,14


In [94]:
info["recommend"] = recs["recommend"][recs["track_id"] == info["track_id"] ]

In [99]:
info.head(20)

,track_id,title,artist,recommend,cluster,cluster_new
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),Bo Carter,0,23,33
1,TRBGPYK128F42796E1,Des Vôtres,Fredericks_ Goldman_ Jones,0,37,14
2,TRBGPJP128E078ED20,Crazy,Aerosmith,1,4,21
3,TRBGPXH128F428C912,Chiove,Rita Chiarelli,0,13,12
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,Nicky B_ Naëlle,0,38,21
5,TRBGPAX12903CEC037,Down South Blues,Sleepy John Estes,0,19,4
6,TRBGPZX128F4285105,Apagon,Estopa,1,32,13
7,TRBGPHO128F4254D34,It Came Upon A Midnight Clear,Cristy Lane,0,28,8
8,TRBGPGF128F4259319,Doin It,Professor Longhair,1,37,14
9,TRBGPCC128F428FF02,Looking In Your Eyes,Suicidal Tendencies,1,20,16


In [97]:
with open(basedir+"trackinfo_new.csv", 'w') as tnf:
    tnf.write(info.to_csv())

In [19]:
preds = tracks[["track_id", "cluster"]]
print len(preds.index)

10000


In [20]:
preds.head()

,track_id,cluster
0,TRBGPHG12903CE6CC3,23
1,TRBGPYK128F42796E1,37
2,TRBGPJP128E078ED20,4
3,TRBGPXH128F428C912,13
4,TRBGPSV12903CA9C25,38


##Combine dfs

In [21]:
trackclusterrecs = pd.merge(recs, preds, how='outer', on="track_id")

In [22]:
trackclusterrecs.head()

,track_id,recommend,cluster
0,TRBGPHG12903CE6CC3,0,23
1,TRBGPYK128F42796E1,0,37
2,TRBGPJP128E078ED20,1,4
3,TRBGPXH128F428C912,0,13
4,TRBGPSV12903CA9C25,1,38


##Bring in track title and artist names

In [23]:
with open(basedir + "titles_csv.csv", 'r') as tf:
    titles = pd.read_csv(tf, index_col=0)

In [24]:
titles.head()

,track_id,title,artist_id,artist,song_id
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),ARFXRHR1187B98FF09,Bo Carter,SORUUEV12A58A7B9FA
1,TRBGPYK128F42796E1,Des Vôtres,ARQDTOS12086C11443,Fredericks_ Goldman_ Jones,SOEZDOH12A8AE4787A
2,TRBGPJP128E078ED20,Crazy,AR12F2S1187FB56EEF,Aerosmith,SOOOWIC12A6701C7E5
3,TRBGPXH128F428C912,Chiove,AR7LIU31187B98EF11,Rita Chiarelli,SOLSWBA12A8C141B9A
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,ARPGCHN1187B9A2831,Nicky B_ Naëlle,SOHNJQL12AB018CC5C


In [25]:
titles["title"] = titles["title"].str.replace(",","")
titles["title"] = titles["title"].str.replace("'", "")
titles["title"].head(10)

0                     The Law Gonna Step On You (1931)
1                                           Des Vôtres
2                                                Crazy
3                                               Chiove
4    Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...
5                                     Down South Blues
6                                               Apagon
7                        It Came Upon A Midnight Clear
8                                              Doin It
9                                 Looking In Your Eyes
Name: title, dtype: object

In [26]:
titles["artist"] = titles["artist"].str.replace(",","")
titles["artist"] = titles["artist"].str.replace("'", "")
titles["artist"].head()

0                     Bo Carter
1    Fredericks_ Goldman_ Jones
2                     Aerosmith
3                Rita Chiarelli
4               Nicky B_ Naëlle
Name: artist, dtype: object

In [27]:
names = titles[["track_id", "title", "artist"]]

In [28]:
names.head()

,track_id,title,artist
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),Bo Carter
1,TRBGPYK128F42796E1,Des Vôtres,Fredericks_ Goldman_ Jones
2,TRBGPJP128E078ED20,Crazy,Aerosmith
3,TRBGPXH128F428C912,Chiove,Rita Chiarelli
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,Nicky B_ Naëlle


##Make final df of track info for recommendations

In [29]:
trackinfo = pd.merge(names, trackclusterrecs, how='outer', on="track_id")

In [30]:
trackinfo.head()

,track_id,title,artist,recommend,cluster
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),Bo Carter,0,23
1,TRBGPYK128F42796E1,Des Vôtres,Fredericks_ Goldman_ Jones,0,37
2,TRBGPJP128E078ED20,Crazy,Aerosmith,1,4
3,TRBGPXH128F428C912,Chiove,Rita Chiarelli,0,13
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,Nicky B_ Naëlle,1,38


In [31]:
with open(basedir+"trackinfo.csv", 'w') as twf:
    twf.write(trackinfo.to_csv())